In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from energyquantified import EnergyQuantified
from energyquantified.metadata import ContractPeriod
from statsmodels.tsa.seasonal import seasonal_decompose

from historicl_movement_functions.ttf_download_hisotry import ttf_download_hisotry_func
from datetime import datetime
from long_term_volatility_calc import *


In [2]:
api_key = 'ebbdc4-a81661-5dfb18-e24b2e'
http_proxy  = "http://proxy.spp.sk:8080"
https_proxy  = "http://proxy.spp.sk:8080"

proxies = {"http"  : http_proxy,
            "https"  : https_proxy}

eq = EnergyQuantified(api_key=api_key, proxies= proxies)
eq.is_api_key_valid()

True

In [3]:
fwd_time_series = eq.ohlc.load(
   'NL Futures Natural Gas EUR/MWh ICE-TTF OHLC',
   begin=date(2010, 1, 1),
   end=date(2020, 1, 1),
   period=ContractPeriod.MONTH  # Only monthly contracts
)

df = fwd_time_series.to_dataframe()
# forward_prices['delivery'] = pd.to_datetime(forward_prices['delivery'], errors='coerce')

# # 3) add year and month columns
# forward_prices['year'] = forward_prices['delivery'].dt.year
# forward_prices['month'] = forward_prices['delivery'].dt.month_name()   # use .dt.month_name() for names instead or dt.month 

In [4]:
df

,traded,period,front,delivery,open,high,low,close,settlement,volume,open_interest
0,2010-01-04,month,1,2010-02-01,NaN,NaN,NaN,NaN,13.80,90.0,0.0
1,2010-01-04,month,2,2010-03-01,NaN,NaN,NaN,NaN,13.71,0.0,0.0
2,2010-01-04,month,3,2010-04-01,NaN,NaN,NaN,NaN,13.34,0.0,0.0
3,2010-01-05,month,1,2010-02-01,NaN,NaN,NaN,NaN,13.97,150.0,0.0
4,2010-01-05,month,2,2010-03-01,NaN,NaN,NaN,NaN,13.72,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
88640,2019-12-31,month,44,2023-09-01,NaN,NaN,NaN,NaN,16.37,5.0,6664.0
88641,2019-12-31,month,45,2023-10-01,NaN,NaN,NaN,NaN,16.85,5.0,6889.0
88642,2019-12-31,month,46,2023-11-01,NaN,NaN,NaN,NaN,17.63,5.0,6889.0
88643,2019-12-31,month,47,2023-12-01,NaN,NaN,NaN,NaN,17.74,5.0,6889.0


In [5]:
lt_vol = compute_long_term_vol_from_df(df, day_count=252, winsor=0.01)
print(f"long_term_vol = {lt_vol:.4f}  # annualized")

PeriodIndex(['2010-01', '2010-01', '2010-01', '2010-01', '2010-01', '2010-01',
             '2010-01', '2010-01', '2010-01', '2010-01',
             ...
             '2019-12', '2019-12', '2019-12', '2019-12', '2019-12', '2019-12',
             '2019-12', '2019-12', '2019-12', '2019-12'],
            dtype='period[M]', name='traded', length=2575)
long_term_vol = 0.1709  # annualized
